# scRNA-seq and scATAC-seq integration analysis
2026.02.20 @carushi 

## Preprocess
* Download scRNA-seq and scATAC-seq metadata from GEO
    * https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE273197
* scRNA-seq data (normalized)
    * GSE273197_scrna_X.mtx.gz (normalized)
    * GSE273197_scrna_obs.csv.gz
    * GSE273197_scrna_var.csv.gz
* scRNA-seq data (raw count)
    * GSE273197_scrna_X_raw.mtx.gz
    * GSE273197_scrna_obs_raw.csv.gz
    * GSE273197_scrna_var_raw.csv.gz
* scATAC-seq data
    * GSE273197_scatac_snap_obj_clust.rds.gz
* Integrated data
    * GSE273197_scrna_atac_integrated.h5Seurat
* Place the files under data/geo_metadata

## Preprocess 2
* Get whitelist data provided by 10x Genomics
    * scRNA-seq: 3M-february-2018.txt.gz
    * scATAC-seq: 737K-cratac-v1.txt
    * See https://kb.10xgenomics.com/s/article/115004506263-What-is-a-barcode-inclusion-list-formerly-barcode-whitelist
* Download CellMarker 2.0 annotation file
    * Open Cell_marker_All.xlsx using Excel
    * Save as txt file
    * nkf -Lu all_cell_marker.txt > all_cell_marker_lu.txt (for UNIX)
    * mv all_cell_marker_lu.txt ../data/gene/
    * run parse_cell_marker.R
* Download Mat (gene set) files
    * Download from xxxxxx and plate them under /data/gene/ 

## scRNA-seq analysis (original)
* Run STARsolo for alignment

## scRNA-seq analysis by Scanpy
* Start from here
* gunzip ../data/geo_metadata/ files
* Run scRNA_scanpy.ipynb (please follow Data setup 2)
* scarmadillo_filt_scobj.pyn and scarmadillo_filt_modf_scobj.pyn will be created

## Data conversion
* Convert scRNA-seq data into csv and create seurat_scrna_integrated.rds

In [11]:
!python ../scripts/alignment/miscs/write_scanpy_obj.py scarmadillo_filtered_modf_scobj.pyn scarmadillo_filtered
!Rscript ../scripts/alignment/miscs/read_scRNAseq.R

 要求されたパッケージ SeuratObject をロード中です 
 要求されたパッケージ sp をロード中です 

 次のパッケージを付け加えます: ‘SeuratObject’ 

 以下のオブジェクトは ‘package:base’ からマスクされています:

    intersect, t

 警告メッセージ: 
1:  パッケージ ‘Seurat’ はバージョン 4.3.3 の R の下で造られました  
2:  パッケージ ‘SeuratObject’ はバージョン 4.3.3 の R の下で造られました  
 [1]  0  3  6  2  5  8 14  7  4 10  9 11 13 12  1 15
[1] "../data/bam/bam_p_cd4/305264Solo.out/Gene/filtered"
[1] 33472  4855
[1] 33472     2
[1] 4855   11
[1] 4855    1
                V1          barcode batch  cov n_counts_spike n_counts
1 AAACCCACAGCACCCA AAACCCACAGCACCCA     0 3151              0     3149
2 AAACCCACATGACAAA AAACCCACATGACAAA     0 1999              0     1999
3 AAACCCAGTCTGTCAA AAACCCAGTCTGTCAA     0 6498              0     6497
4 AAACCCATCCACGTGG AAACCCATCCACGTGG     0 2628              0     2627
5 AAACGAAAGACCATAA AAACGAAAGACCATAA     0 3841              0     3840
6 AAACGAAAGTTAGAAC AAACGAAAGTTAGAAC     0 7060              0     7060
      tsne1       tsne2     umap1     umap2 cluster_louvain cluster_

## scATAC-seq analysis (original)
### Run STARsolo alignment and SnapTools


In [12]:
!bash ../scripts/alignment/snaptools_preprocess.sh
# bash ../scripts/alignment/snaptools_preprocess_pub.sh
# bash ../scripts/alignment/snaptools_preprocess_relaxed.sh

ls: ../data/bam_pub_genome//*.out.bam: No such file or directory


## scATAC-seq analysis by SnapATAC
### Set up conda environments
* genome - insntall MACS2, snapatac, and HOMER at ~/miniconda/env/genome/bin
* You can change this setting defined in multi_sc_base.R

In [ ]:
Rscript ../scripts/sc_clustering/snap_basic_stats.r make_list file_list.txt
Rscript ../scripts/sc_clustering/snap_basic_stats.r plot file_list.txt 500 2.35

### Relaxed setting
abolished

In [ ]:
# Rscript ../scripts/sc_clustering/snap_basic_stats.r make_list file_list.txt
# Rscript ../scripts/sc_clustering/snap_basic_stats.r plot file_list.txt 1500 2.35

In [ ]:
### Filter out unreliable cells
Rscript ../scripts/sc_clustering/snap_atac_clustering.R snap_obj_filtered.rds

In [ ]:
Rscript ../scripts/sc_clustering/snap_obj_analysis.R snap_obj_clust.rds

## Seurat CCA integration and cell type annotation


In [ ]:
### Clustering and annotation
Rscript ../scripts/sc_clustering/coembedding_for_cell_typing.R

In [ ]:
Rscript ../scripts/sc_clustering/seurat_obj_analysis.R final_pseudo_bulk_atac.rds
Rscript ../scripts/sc_clustering/seurat_obj_analysis.R final_pseudo_bulk_rna.rds
Rscript ../scripts/sc_clustering/seurat_obj_analysis.R final_coembed.rds

## Extra

In [ ]:
Rscript extract_barcode.R
bash run_cellsnp.sh